In [2]:
#Checking if GPU is running or not

!nvidia-smi

Sun Jan 26 08:34:58 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import string
import re

from keras.models import Model
from keras.layers import Input, LSTM, Dense

import warnings
warnings.filterwarnings("ignore")

## 1. Loading data

In [4]:
df = pd.read_csv("/content/hindi_english_parallel.csv")

In [5]:
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [6]:
ndf = df.dropna(axis=0)

In [7]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1555574 entries, 0 to 1561839
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   hindi    1555574 non-null  object
 1   english  1555574 non-null  object
dtypes: object(2)
memory usage: 35.6+ MB


In [8]:
ndf.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [9]:
ndf.iloc[0,:]

,0
hindi,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
english,Give your application an accessibility workout


## 2. Data cleaning

In [10]:
def data_cleaning(text):
    sample = text
    sample = [char for char in text if char not in string.punctuation]
    return sample

input_text = ndf['hindi'][:10000].apply(data_cleaning)
target_text = ndf['english'][:10000].apply(data_cleaning)

## 3. Tokenizing the data

In [11]:
input_char = set()
target_char = set()

# total unique words in hindi corpus
for line in input_text:
    for char in line:
        if char not in input_char:
            if char not in string.punctuation:
                input_char.add(char)

# total unique words in english corpus
for line2 in target_text:
    for char2 in line2:
        if char2 not in target_char:
            if char2 not in string.punctuation:
                target_char.add(char2)

In [12]:
# find chars to encode and max encoder seq length
input_characters = sorted(list(input_char))
target_characters = sorted(list(target_char))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_text])
max_decoder_seq_length = max([len(txt) for txt in target_text])

print('Number of samples:', len(input_text))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 125
Number of unique output tokens: 67
Max sequence length for inputs: 272
Max sequence length for outputs: 224


In [13]:
# indexing the input and target tokens
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char,i) for i, char in enumerate(target_characters)])

In [14]:
# encoder and decoder input data
encoder_input_data = np.zeros(
    (len(input_text), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_text), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [15]:
# shape of input encoder data
encoder_input_data.shape

(10000, 272, 125)

In [16]:
# shape of output decoder data
decoder_target_data.shape

(10000, 224, 67)

In [17]:
# create encoder and decoder train the translation model
for i, (input_text, target_text) in enumerate(zip(input_text, target_text)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

## 4. Modeling and evaluation

In [18]:
batch_size = 64  # Batch size for training.
epochs = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.

In [19]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [20]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [21]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, None, 125)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None, 67)       │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        391,168 │ input_layer[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, None, 256),    │        331,776 │ input_layer_1[0][0],   │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, None, 67)       │         17,219 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 740,163 (2.82 MB)

 Trainable params: 740,163 (2.82 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 66ms/step - accuracy: 0.8907 - loss: 0.7057 - val_accuracy: 0.9549 - val_loss: 0.2105
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 0.9329 - loss: 0.2982 - val_accuracy: 0.9549 - val_loss: 0.1906
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 51ms/step - accuracy: 0.9313 - loss: 0.2778 - val_accuracy: 0.9520 - val_loss: 0.1885
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 51ms/step - accuracy: 0.9304 - loss: 0.2680 - val_accuracy: 0.9528 - val_loss: 0.1828
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9318 - loss: 0.2583 - val_accuracy: 0.9544 - val_loss: 0.1749
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 49ms/step - accuracy: 0.9336 - loss: 0.2511 - val_accuracy: 0.9552 - val_loss: 0.1723
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 52ms/step - accuracy: 0.9353 - loss: 0.2452 - val_accuracy: 0.9568 - val_loss: 0.1680
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - accuracy: 0.9379 - loss: 0.2309 - val

In [23]:
model.save('translation.keras')